# Пользовательские демографические группы
Ноутбук с примерами формирования пользовательских демографических групп

Общие принципы формирования демографических групп:
- формируются на основе значений каталога демографических переменных
- Доступные операторы:
    ```
    = (равенство)
    != (неравенство)
    > (больше)
    < (меньше)
    AND (И)
    OR (ИЛИ)
    IN() (вхождение в список значений)
    NOTIN() (невхождение в список значений)
    ```
- если в демографической группе, передаваемой в demo_filter, есть отбор по __возрасту__, то переменной __ages__ должно быть присвоено значение None
    
    ```python
    ages = None
    ```
    

- если в демографической группе, передаваемой в demo_filter, есть отбор по __численности населения города__, то переменной __population__ должно быть присвоено значение None

    ```python
    population = None
    ```

Рассмотрим подробнее на примере следующих демографических групп:
1. Мужчины 35-44 лет
2. Мужчины 18-24 лет или женщины 25-34 лет
3. Все 20-56 лет
4. 25-54 не руководители
5. Жители городов 700+ с доходом BC

## Инициализация
Загрузим библиотеки, которые нам помогут обращаться к API Responsum и работать с данными

In [1]:
%reload_ext autoreload
%autoreload 2

import os
import re
import json
import datetime
import time
import pandas as pd
from pathlib import Path
from IPython.display import JSON

from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

In [2]:
#Зададим тип установки
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

# создадим объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)

print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Объектов в media-каталоге: 4716


## Список демографических переменных
Для формирования синтаксиса демографических групп, нам понадобится каталог демографических переменных

Загрузим каталог (подробнее работа с каталогами описана в [ноутбуке](catalogs.ipynb#catalogs_site_avito))

In [3]:
demo_cat = rcats.get_demo()
demo_cat

,varId,varName,varTitle,categories,from,to,catNum,catTitle
0,106,SEX,Пол,"[{'varId': 106, 'catNum': 1, 'title': 'Мужчины...",None,None,1,Мужчины
1,106,SEX,Пол,"[{'varId': 106, 'catNum': 1, 'title': 'Мужчины...",None,None,2,Женщины
2,107,CITY,Город проживания до 01.02.2019,"[{'varId': 107, 'catNum': 1, 'title': 'Москва'...",None,None,1,Москва
3,107,CITY,Город проживания до 01.02.2019,"[{'varId': 107, 'catNum': 1, 'title': 'Москва'...",None,None,2,Санкт-Петербург
4,107,CITY,Город проживания до 01.02.2019,"[{'varId': 107, 'catNum': 1, 'title': 'Москва'...",None,None,27,Казань
...,...,...,...,...,...,...,...,...
93,719,CITY_BM,Город проживания,"[{'varId': 719, 'catNum': 1, 'title': 'Большая...",None,None,2,Санкт-Петербург
94,719,CITY_BM,Город проживания,"[{'varId': 719, 'catNum': 1, 'title': 'Большая...",None,None,27,Казань
95,719,CITY_BM,Город проживания,"[{'varId': 719, 'catNum': 1, 'title': 'Большая...",None,None,47,Екатеринбург
96,719,CITY_BM,Город проживания,"[{'varId': 719, 'catNum': 1, 'title': 'Большая...",None,None,64,Новосибирск


Для удобства можно выгрузить каталог в Excel и работать с ним там. Файл созраняется в директорию с ноутбуком.

In [4]:
writer = pd.ExcelWriter('demo_cat.xlsx')
demo_cat.to_excel(writer, 'demo_cat', index=False)
writer.save()


## 1. Мужчины 35-44 лет

Из каталога видим, что для отбора мужчин нам нужно значение 1 переменной SEX
```
    SEX = 1
```
Возрастной группе 35-44 соответствует значение 4 переменной AGE_GROUPS
```
    AGE_GROUPS = 4
```
Объединяем выражения через оператор AND
```
    SEX = 1 AND AGE_GROUPS = 4
```

### Задаем параметры расчета

__NB__ Если в демографической переменной используется __возраст__ (как в нашем случае), то переменной __ages__ должно быть присвоено значение __None__

In [5]:
# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем Типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# задаем Население, например, Россия 100+
population = rcats.get_population('Russia100+')

# присваиваем переменной ages значение None
ages = None

# Провеяем, что значения парамеров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes, population, ages)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Дата/время расчета: 2021-03-23 10:44:49
Тип установки: mobile
Начало периода: 2020-05-01
Конец периода: 2020-05-31
Типы пользования Интернетом: [1, 2, 3, 4]
Население: [1, 2, 3]
Возрастные группы: None
Объектов в media-каталоге: 4716


### Формируем задание на расчет

В demo_filter передаем сформированный синтакисиc переменной в кавычках

In [6]:
# Задаем название проекта для отображения в DataFrame
project_name = 'M35-44'

# Фильтр по медиа отсутствует
media_filter = None

# Задаем демографическую группу в demo_filter
demo_filter = "SEX = 1 AND AGE_GROUPS = 4"

# Задаем список статистик для расчета
statistics=["UnwReach"]

# Указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
}

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))
# Получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

Задача <62763c07-2dd5-46a7-a141-c1bb5e84e87e> поступила в обработку
Расчет задачи [= = = = ] время расчета: 0:00:13.502242


,prj_name,stat_unwreach
0,M35-44,2069.0


## 2. Мужчины 18-24 лет или женщины 25-34 лет

Возвращаемся к каталогу демографических переменных. Для отбора мужчин 18-24 лет нам нужно значение 1 переменной SEX и значение 2 переменной AGE_GROUPS
```
SEX = 1 AND AGE_GROUPS = 2
```
Для отбора женщин 25-34 лет синтаксис будет следующим:
```
SEX = 2 AND AGE_GROUPS = 3
```
Объединяем через оператор OR
```
(SEX = 1 AND AGE_GROUPS = 2) OR (SEX = 2 AND AGE_GROUPS = 3)
```

### Задаем параметры расчета

__NB__ Если в демографической переменной используется __возраст__ (как в нашем случае), то переменной __ages__ должно быть присвоено значение __None__

In [7]:
# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем Типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# задаем Население, например, Россия 100+
population = rcats.get_population('Russia100+')

# присваиваем переменной ages значение None
ages = None

# Провеяем, что значения парамеров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes, population, ages)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Дата/время расчета: 2021-03-16 12:08:50
Тип установки: mobile
Начало периода: 2020-05-01
Конец периода: 2020-05-31
Типы пользования Интернетом: [1, 2, 3, 4]
Население: [1, 2, 3]
Возрастные группы: None
Объектов в media-каталоге: 4716


### Формируем задание на расчет

В demo_filter передаем сформированный синтакисиc переменной в кавычках

In [7]:
# Задаем название проекта для отображения в DataFrame
project_name = 'M18-24_F25-34'

# Фильтр по медиа отсутствует
media_filter = None

# Задаем демографическую группу в demo_filter
demo_filter = "(SEX = 1 AND AGE_GROUPS = 2) OR (SEX = 2 AND AGE_GROUPS = 3)"

# Задаем список статистик для расчета
statistics=["UnwReach"]

# Указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
}

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))
# Получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

Задача <27538ab8-17ee-4abf-acdc-8b6238000a14> поступила в обработку
Расчет задачи [= = = = = = ] время расчета: 0:00:20.243647


,prj_name,stat_unwreach
0,M18-24_F25-34,2771.0


## 3. Все 20-56 лет

Возраст 20-56 лет не получится собрать из переменной AGE_GROUPS, нам понадобится переменная AGE

Зададим диапазон возраста.
Можно использовать только операторы сравнения "больше"(>) и "меньше"(<), поэтому наше выражение будет выглядеть так:
```
AGE > 19 AND AGE < 57
```

### Задаем параметры расчета

__NB__ Если в демографической переменной используется __возраст__ (как в нашем случае), то переменной __ages__ должно быть присвоено значение __None__

In [8]:
# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем Типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# задаем Население, например, Россия 100+
population = rcats.get_population('Russia100+')

# присваиваем переменной ages значение None
ages = None

# Провеяем, что значения парамеров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes, population, ages)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Дата/время расчета: 2021-03-23 10:45:43
Тип установки: mobile
Начало периода: 2020-05-01
Конец периода: 2020-05-31
Типы пользования Интернетом: [1, 2, 3, 4]
Население: [1, 2, 3]
Возрастные группы: None
Объектов в media-каталоге: 4716


### Формируем задание на расчет

В demo_filter передаем сформированный синтакисиc переменной в кавычках

In [9]:
# Задаем название проекта для отображения в DataFrame
project_name = 'All20-56'

# Фильтр по медиа отсутствует
media_filter = None

# Задаем демографическую группу в demo_filter
demo_filter = "(AGE > 19 AND AGE < 57)"

# Задаем список статистик для расчета
statistics=["UnwReach"]

# Указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
}

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))
# Получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

Задача <27cbf234-8488-41fd-815f-9ac7f01533c9> поступила в обработку
Расчет задачи [= = = = = = ] время расчета: 0:00:19.734733


,prj_name,stat_unwreach
0,All20-56,12254.0


## 4. 25-54 не руководители

Для отбора возрастной группы 25-54 нам понадобятся значения 3, 4 и 5 переменной AGE_GROUPS. Чтобы не мучиться с написанием трех условий через OR, воспользуемся оператором IN
```
AGE_GROUPS IN (3, 4, 5)
```
Условие "не руководители" можно также коротко задать при помощи оператора неравенства "!="
```
OCCUPATION2 != 1
```
Объединяем через оператор AND
```
AGE_GROUPS IN (3, 4, 5) AND OCCUPATION2 != 1
```

### Задаем параметры расчета

__NB__ Если в демографической переменной используется __возраст__ (как в нашем случае), то переменной __ages__ должно быть присвоено значение __None__

In [11]:
# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем Типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# задаем Население, например, Россия 100+
population = rcats.get_population('Russia100+')

# присваиваем переменной ages значение None
ages = None

# Провеяем, что значения парамеров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes, population, ages)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Дата/время расчета: 2021-03-16 12:10:16
Тип установки: mobile
Начало периода: 2020-05-01
Конец периода: 2020-05-31
Типы пользования Интернетом: [1, 2, 3, 4]
Население: [1, 2, 3]
Возрастные группы: None
Объектов в media-каталоге: 4716


### Формируем задание на расчет

В demo_filter передаем сформированный синтакисиc переменной в кавычках

In [12]:
# Задаем название проекта для отображения в DataFrame
project_name = '25-54_not_dir'

# Фильтр по медиа отсутствует
media_filter = None

# Задаем демографическую группу в demo_filter
demo_filter = "AGE_GROUPS IN (3, 4, 5) AND OCCUPATION2 != 1"

# Задаем список статистик для расчета
statistics=["UnwReach"]

# Указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
}

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))
# Получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

Задача <02e471e3-9a48-41c1-8d6e-62f445139e69> поступила в обработку
Расчет задачи [= = = = = ] время расчета: 0:00:16.807679


,prj_name,stat_unwreach
0,25-54_not_dir,9086.0


## 5. Жители городов 700+ с доходом BC

Размер города 700+ зададим при помощи переменной CITY_TYPE2
```
CITY_TYPE2 = 1
```
Для отбора респондентов с доходом BC (средний и выше среднего) нам потребуется переменная INCOME_GROUPS
```
INCOME_GROUPS IN (2, 3)
```
Объединим выражения через оператор AND
```
CITY_TYPE2 = 1 AND INCOME_GROUPS IN (2, 3)
```

### Задаем параметры расчета

__NB__ Если в демографической переменной используется __численность населения города__ (как в нашем случае), то переменной __population__ должно быть присвоено значение __None__

In [13]:
# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем Типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# выбираем города с населением 100 тыс и более
population = rcats.get_population('Russia100+')

# задаем Возврастные группы, нас интересует 12+
ages = rcats.get_age_groups('12+')

# Провеяем, что значения парамеров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes, population, ages)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Дата/время расчета: 2021-03-16 12:10:44
Тип установки: mobile
Начало периода: 2020-05-01
Конец периода: 2020-05-31
Типы пользования Интернетом: [1, 2, 3, 4]
Население: [1, 2, 3]
Возрастные группы: [1, 2, 3, 4, 5, 6, 7]
Объектов в media-каталоге: 4716


### Формируем задание на расчет

В demo_filter передаем сформированный синтакисиc переменной в кавычках

In [14]:
# Задаем название проекта для отображения в DataFrame
project_name = 'All700+BC'

# Фильтр по медиа отсутствует
media_filter = None

# Задаем демографическую группу в demo_filter
demo_filter = "CITY_TYPE2 = 1 AND INCOME_GROUPS IN (2, 3)"

# Задаем список статистик для расчета
statistics=["UnwReach"]

# Указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
}

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))
# Получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

Задача <e8dae1f1-98bd-4832-8952-cebad76c2873> поступила в обработку
Расчет задачи [= = = = = = = ] время расчета: 0:00:23.467137


,prj_name,stat_unwreach
0,All700+BC,8810.0
